In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [24]:
#names_df = pd.read_csv("Resources/yob1978.txt", delimiter =",", names=["Name", "Gender", "YearOfBirth"],dtype={"Name":"str","Gender":"str", "YearOfBirth":np.int32})

start_year=1881
end_year=2019
yearlist= list(range(start_year,end_year,1))
#print(yearlist)
year_df=pd.DataFrame(yearlist)
year_df["dummy_col"]=1
year_df.set_index("dummy_col", inplace=True)
filename_list=[]
for file in os.listdir("Resources"):
    if file.endswith(".txt") and int(file[3:7]) >= start_year:
        df =  pd.read_csv("Resources/"+file, delimiter =",", names=["Name", "Gender", "CountOfNames"],dtype={"Name":"str","Gender":"str", "CountOfNames":np.int32})
        df["Year"] = int(file[3:7])
        
        df = df[["Year", "Name", "Gender", "CountOfNames"]]
        filename_list.append(df)
        
names_df = pd.concat(filename_list, axis=0, ignore_index=True)
names_df.sort_values(by=["Name","Gender","Year"], inplace=True)
unique_names_list = names_df.drop_duplicates(subset=["Name", "Gender"])[["Name", "Gender"]]
unique_names_list["dummy_col"]=1
unique_names_list.set_index("dummy_col", inplace=True)

final_data_set = unique_names_list.join(year_df,on="dummy_col",how = "outer")
final_data_set.count()
final_data_set.rename(columns={0:"Year"}, inplace=True)
final_data_set.count()

Name      15065736
Gender    15065736
Year      15065736
dtype: int64

In [56]:
### We have a dataset with entries for all names for all years of analysis. 
### We need to now update the counts of each row with the actual values.

data_set_with_counts = final_data_set.merge(names_df, on=["Name", "Gender", "Year"], how = "outer")
data_set_with_counts.fillna(0, inplace = True)
data_set_with_counts["Percent_Change"] = data_set_with_counts["CountOfNames"].pct_change() * 100
final_data_set.count()

Name      15065736
Gender    15065736
Year      15065736
dtype: int64

In [57]:
data_set_with_counts.count()

Name              15065736
Gender            15065736
Year              15065736
CountOfNames      15065736
Percent_Change     2353284
dtype: int64

In [58]:
names_df.head(5)

,Year,Name,Gender,CountOfNames
1584216,2007,Aaban,M,5
1652500,2009,Aaban,M,6
1683983,2010,Aaban,M,9
1716670,2011,Aaban,M,11
1750571,2012,Aaban,M,11


In [59]:
data_set_with_counts.loc[(data_set_with_counts["Name"] =="Aaban") & (data_set_with_counts["Year"]>= 2005)]

,Name,Gender,Year,CountOfNames,Percent_Change
124,Aaban,M,2005,0.0,NaN
125,Aaban,M,2006,0.0,NaN
126,Aaban,M,2007,5.0,inf
127,Aaban,M,2008,0.0,-100.000000
128,Aaban,M,2009,6.0,inf
129,Aaban,M,2010,9.0,50.000000
130,Aaban,M,2011,11.0,22.222222
131,Aaban,M,2012,11.0,0.000000
132,Aaban,M,2013,14.0,27.272727
133,Aaban,M,2014,16.0,14.285714


In [61]:
# line plot function
def getPlot(name,gender,center):
    # get a dataframe for just that name and gender
    plot_df = data_set_with_counts.loc[(names_df['Name']==name) & (names_df['Gender']==gender) ]
    
    # set the series to plot
    x_axis = plot_df['Year']
    values = plot_df['CountOfNames']
    # create the plot
    plt.figure(figsize=[15,5])
    plt.plot(x_axis,values,color = 'coral',marker = 'o')
    
    # set the axes limits and ticks
    plt.ylim(0,max(values)*1.05)
    plt.xlim(center-30,center+30)
    plt.xticks(np.arange(center-30,center+30,10))
    # draw a horizontal line at the center
    plt.vlines(center,0,max(values)*1.05, color = 'blue')
    
    # other chart elements
    plt.title(f'Popularity of "{name}" ({gender})')
    plt.ylabel('Count')
    plt.xlabel('Year')
    plt.grid()
    plt.savefig(f'plots/{name}-{gender}.png', dpi=600)

In [62]:
def getPlot2(name,gender,center):
    # get a dataframe for that year and gender
    plot_df = data_set_with_counts.loc[(data_set_with_counts['Year']==center) & (data_set_with_counts['Gender']==gender)]
    plot_df1 = data_set_with_counts.loc[(sorted_df['Year']==center) & (sorted_df['Gender']==gender) & (sorted_df['Name']==name)]

    # set the series to plot
    x_axis = plot_df['CountOfNames']
    values = plot_df['Percentage_Change']
    x_focus = plot_df1['CountOfNames']
    value_focus = plot_df1['Percentage_Change']
    
    # create the plot
    plt.figure(figsize=[15,5])
    plot1 = plt.scatter(x_axis,values,color = 'coral',marker = 'o')
    Plot2 = plt.scatter(x_focus,value_focus, color = 'blue',marker = 's')

    # other chart elements
    plt.title(f'Popularity of "{name}" ({gender}) in {center}')
    plt.ylabel('Pct Change from Prior Year')
    plt.xlabel('Count')
    plt.xlim(0,20000)
    plt.grid()
    #plt.savefig(f'plots/{name}-{gender}.png', dpi=600)

In [63]:
name = input('Name? >>> ')
gender = input('Gender? M/F >>> ')
center = int(input('Year? >>> '))

Name? >>> Aaban
Gender? M/F >>> M
Year? >>> 2008


In [64]:
getPlot(name,gender,center)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match